## 0. Import packages

In [450]:
# data processing
import pandas as pd 
import numpy as np

# database agent
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

# env variables
import os
from dotenv import load_dotenv
load_dotenv()
database_credentials = {
    "username": os.getenv("DB_USERNAME"),
    "password": os.getenv("DB_PASSWORD")
}

# sys to add parent folder to path
import sys
sys.path.append("..")

from src.support_database_connection import connect_to_database

## 1. Introduction

The purpose of this notebook is to explore the given datasets api_foursquare, df_aemet_final and municipios and to decide what columns to select, separate in a different table or to remove, to finally load them as tables to a database in PostgreSQL.

## 2. Exploration

### 2.1 Foursquare

Load and inspect the first rows of the dataset.

In [451]:
foursquare_df = pd.read_csv("../data/api_foursquare.csv", index_col = 0)
foursquare_df.head()

,municipio,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude
0,ajalvir,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341
1,ajalvir,Park,4f54f7bfe4b036244d02685b,VeryLikelyOpen,1724.0,/v3/places/4f54f7bfe4b036244d02685b,Parque Infantil,NaN,40.543867,-3.464773
2,ajalvir,Park,7d4ef99ca2dd4789f032a90c,LikelyOpen,333.0,/v3/places/7d4ef99ca2dd4789f032a90c,Chiquitin,Ajalvir Madrid,40.531662,-3.482835
3,alamo-el,Park,53de1144498e8fd8d8cb3537,LikelyOpen,118.0,/v3/places/53de1144498e8fd8d8cb3537,Isla Perejil,28607 El Alamo Comunidad de Madrid,40.230993,-3.991942
4,alamo-el,Park,51a8f451498ecb8334de37df,LikelyOpen,582.0,/v3/places/51a8f451498ecb8334de37df,Parque de Madera,NaN,40.234636,-3.995166


Just looking at the values themselves does not give much information. Let's check datatypes and unique values.

In [452]:
foursquare_df.info()
print("\n-----")
for column in foursquare_df:
    print(f"Number of unique values per {column}: ", foursquare_df[column].nunique())

<class 'pandas.core.frame.DataFrame'>
Index: 907 entries, 0 to 906
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   municipio      907 non-null    object 
 1   category       907 non-null    object 
 2   fsq_id         907 non-null    object 
 3   closed_bucket  907 non-null    object 
 4   distance       907 non-null    float64
 5   link           907 non-null    object 
 6   name           907 non-null    object 
 7   address        778 non-null    object 
 8   latitude       907 non-null    float64
 9   longitude      907 non-null    float64
dtypes: float64(3), object(7)
memory usage: 77.9+ KB

-----
Number of unique values per municipio:  96
Number of unique values per category:  10
Number of unique values per fsq_id:  822
Number of unique values per closed_bucket:  3
Number of unique values per distance:  724
Number of unique values per link:  822
Number of unique values per name:  764
Number of unique va

Observing value counts, there are fewer unique values of name and address, which means not all values are unique or there are missing values. Let's check the missing values and also group them by name to count their occurrences.

In [453]:
foursquare_df.isna().sum()

municipio          0
category           0
fsq_id             0
closed_bucket      0
distance           0
link               0
name               0
address          129
latitude           0
longitude          0
dtype: int64

Address is the only variable with missing values, and besides, they are not as many as to be able to explain why there are so much fewer unique values than expected for unique foursquare places. Therefore the need to check the value counts for each of these two features.

In the case of name, it might seem that in some cases the value inside is another category or subcategory, rather than a name.

In [454]:
foursquare_df["name"].value_counts().reset_index()[:15]

,name,count
0,Parque Infantil,19
1,Parque,10
2,Dog Run,6
3,Parque de la Marina,4
4,Parque infantil,3
5,Parque Salvador Allende,3
6,Parque de la Tejera,3
7,Parque de Extremadura,3
8,Parque de Perros,3
9,El Parque,3


For example, for the same name Parque Infantil or Parque Salvador Allende there are several foursquare places. Looking at specific examples, it seems that there are places with generic names, but that belong to a different municipio.

In [455]:
filtro_name_isidro = foursquare_df["name"] == 'Dog Run'
foursquare_df[filtro_name_isidro]

,municipio,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude
614,pozuelo-de-alarcon,Park,4d73f327e278f04d972751b8,LikelyOpen,1196.0,/v3/places/4d73f327e278f04d972751b8,Dog Run,28224 Pozuelo de Alarcón Comunidad de Madrid,40.440103,-3.802633
647,rozas-de-madrid-las,Park,4bf7c792508c0f47c6f83d31,LikelyOpen,318.0,/v3/places/4bf7c792508c0f47c6f83d31,Dog Run,28230 Las Rozas Comunidad de Madrid,40.493295,-3.872065
650,rozas-de-madrid-las,Park,4dac76534df01c19b1b4e626,LikelyOpen,398.0,/v3/places/4dac76534df01c19b1b4e626,Dog Run,28231 Las Rozas Comunidad de Madrid,40.496604,-3.873975
657,rozas-de-madrid-las,Park,4ef0b5645c5ca1ec31296a30,LikelyOpen,748.0,/v3/places/4ef0b5645c5ca1ec31296a30,Dog Run,28230 Las Rozas Comunidad de Madrid,40.496611,-3.883562
662,rozas-de-madrid-las,Park,4cffffe70457b1f7a3d82e78,Unsure,1033.0,/v3/places/4cffffe70457b1f7a3d82e78,Dog Run,28231 Las Rozas Comunidad de Madrid,40.499341,-3.866536
664,rozas-de-madrid-las,Park,4d7e51b4555c59418322d077,LikelyOpen,1180.0,/v3/places/4d7e51b4555c59418322d077,Dog Run,28231 Las Rozas Comunidad de Madrid,40.491096,-3.862172


In [456]:
foursquare_df.query("name == 'Parque San Isidro'")

,municipio,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude
13,alcala-de-henares,Park,4d2da1b012d16a31ed5b2c22,LikelyOpen,726.0,/v3/places/4d2da1b012d16a31ed5b2c22,Parque San Isidro,28800 Alcalá de Henares Comunidad de Madrid,40.487883,-3.361226
276,cubas-de-la-sagra,Park,50092b66e4b0d0fe139abf11,LikelyOpen,813.0,/v3/places/50092b66e4b0d0fe139abf11,Parque San Isidro,Cubas de la Sagra Comunidad de Madrid,40.194303,-3.846270
794,torrejon-de-ardoz,Park,4cd5b1d02944b1f7f8ba5eec,VeryLikelyOpen,556.0,/v3/places/4cd5b1d02944b1f7f8ba5eec,Parque San Isidro,San Isidro Madrid,40.454914,-3.483518


In the case of addresses, it is clear that it is a problem of some addresses being incomplete.

In [457]:
foursquare_df["address"].value_counts().reset_index()

,address,count
0,Comunidad de Madrid,32
1,Madrid Comunidad de Madrid,26
2,Alcalá de Henares Comunidad de Madrid,10
3,Madrid,10
4,28850 Torrejón de Ardoz Comunidad de Madrid,9
...,...,...
486,28803 Alcalá de Henares Comunidad de Madrid,1
487,28800 Alcalá de Henares Comunidad de Madrid,1
488,28805 Alcalá de Henares Comunidad de Madrid,1
489,28806 Alcalá de Henares Comunidad de Madrid,1


Finally, there must be duplicates, given that there are fewer unique values for fsq_id than rows.

In [458]:
foursquare_df[foursquare_df.duplicated("fsq_id",keep=False)].sort_values(by="fsq_id").head(6)

,municipio,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude
714,san-lorenzo-de-el-escorial,Monument,09b5d18763e445d74c68f13a,LikelyOpen,1490.0,/v3/places/09b5d18763e445d74c68f13a,Estación de el Escorial,28280 El Escorial Madrid,40.585308,-4.132391
297,escorial-el,Monument,09b5d18763e445d74c68f13a,LikelyOpen,1333.0,/v3/places/09b5d18763e445d74c68f13a,Estación de el Escorial,28280 El Escorial Madrid,40.585308,-4.132391
765,san-sebastian-de-los-reyes,Park,26daa0ca8eaf4c1b6f4be56c,LikelyOpen,981.0,/v3/places/26daa0ca8eaf4c1b6f4be56c,Parque de Félix Rodríguez de la Fuente,Alcobendas Madrid,40.544395,-3.637006
104,alcobendas,Park,26daa0ca8eaf4c1b6f4be56c,LikelyOpen,497.0,/v3/places/26daa0ca8eaf4c1b6f4be56c,Parque de Félix Rodríguez de la Fuente,Alcobendas Madrid,40.544395,-3.637006
195,braojos,Monument,4a470120ec564dc5ff1fae09,LikelyOpen,72.0,/v3/places/4a470120ec564dc5ff1fae09,Ayuntamiento Braojos de la Sierra,Braojos Madrid,41.039387,-3.642364
779,serna-del-monte-la,Monument,4a470120ec564dc5ff1fae09,LikelyOpen,1735.0,/v3/places/4a470120ec564dc5ff1fae09,Ayuntamiento Braojos de la Sierra,Braojos Madrid,41.039387,-3.642364


After that check, it becomes clear that the town is not necessarily the town it belongs to. It could be that the closest town is actually the town where the location is in, or it could not and that column loses all its value altogether.

The actual town could therefore be extracted from the address, although that requires manual labour.

In [459]:
foursquare_df = foursquare_df.sort_values(by="distance").drop_duplicates(subset="fsq_id", keep="first")

This dataset does not seem to need any database normalization performed to be uploaded as a table. To sum up:
- municipio column: It could be dropped, but it is worth keeping for direct information that cannot be easily got from the coordinates or address without formatting.
- distance: To drop. It provides only information about the distance to the town coordinates. The distance to the town of interest will be a calculated metric when necessary.
- category column: Could be a separate table, but for the time being will stay on the same one. Separating it would just mean creating an id column for the sake of saving the space of repeating a category many times instead of an id, which is a fair point, but not considered enough justification.

### 2.2 Aemet

Load and inspect the first rows of the dataset.

In [460]:
aemet_df = pd.read_csv("../data/df_aemet_final.csv")

aemet_df["fecha"] = pd.to_datetime(aemet_df["fecha"]) 

aemet_df.head()

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la-id28001
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,Riesgo,0,75,Riesgo,SE,12,acebeda-la-id28001
2,2024-08-29 14:00:00,Muy nuboso,23,23,33.0,0.0,0,59,100,0,80,Riesgo,S,14,acebeda-la-id28001
3,2024-08-29 15:00:00,Nubes altas,23,23,35.0,0.0,0,55,Riesgo,0,80,Riesgo,S,17,acebeda-la-id28001
4,2024-08-29 16:00:00,Nubes altas,21,21,34.0,0.0,0,65,Riesgo,0,80,Riesgo,S,20,acebeda-la-id28001


Inspecting the numerical datatypes, we can see that right now many of them do not have a lot of variability. That is explained by the fact that the data is not very extensive, coming from less than 2 days of data. 

In [461]:
aemet_df.describe().T

,count,mean,min,25%,50%,75%,max,std
fecha,7885,2024-08-30 09:28:27.545973504,2024-08-29 11:00:00,2024-08-29 22:00:00,2024-08-30 09:00:00,2024-08-30 20:00:00,2024-08-31 07:00:00,NaN
temp._(°c),7885.0,21.239062,9.0,18.0,20.0,25.0,32.0,4.317739
sen._térmica_(°c),7885.0,21.306024,9.0,18.0,20.0,25.0,34.0,4.43552
racha_máx._(km/h),7885.0,20.584147,1.0,13.0,19.0,26.0,83.0,9.904282
precipitación_(mm),7885.0,0.133621,0.0,0.0,0.0,0.0,29.0,0.997648
nieve_(mm),7885.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
humedad_relativa_(%),7885.0,73.479899,34.0,57.0,76.0,89.0,100.0,18.476987
prob._de_nieve_(%),7885.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
prob._de_tormenta_(%),7885.0,39.031706,0.0,10.0,40.0,65.0,90.0,27.452468
velocidad_del_viento,7885.0,11.524794,0.0,7.0,11.0,15.0,45.0,6.032577


Categorical features seem interesting. Localizacion_id has the same amount of unique values as there are municipios. The features cielo, avisos and direccion viento are interesting.

In [462]:
aemet_df.describe(include="O")

,cielo,prob._precip.__(%),avisos,dirección_viento,localizacion_id
count,7885,7885,7885,7885,7885
unique,23,22,2,9,179
top,Poco nuboso,Riesgo,Sin riesgo,NE,alcorcon-id28007
freq,1442,4126,4296,2276,45


Avisos does not seem to be a very objective data, as seen by the value counts and the head of the first registers, as it shows inconsistent criteria for riesgo.

In [463]:
display(aemet_df[["avisos","cielo"]].value_counts().reset_index().sort_values(by="cielo")[:10])
aemet_df.head()

,avisos,cielo,count
33,Riesgo,Bruma,3
29,Sin riesgo,Bruma,7
2,Sin riesgo,Cielo despejado,643
12,Riesgo,Cielo despejado,219
1,Riesgo,Cubierto,824
6,Sin riesgo,Cubierto,493
28,Sin riesgo,Cubierto con lluvia,8
17,Riesgo,Cubierto con lluvia,80
22,Sin riesgo,Cubierto con lluvia escasa,34
13,Riesgo,Cubierto con lluvia escasa,196


,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la-id28001
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,Riesgo,0,75,Riesgo,SE,12,acebeda-la-id28001
2,2024-08-29 14:00:00,Muy nuboso,23,23,33.0,0.0,0,59,100,0,80,Riesgo,S,14,acebeda-la-id28001
3,2024-08-29 15:00:00,Nubes altas,23,23,35.0,0.0,0,55,Riesgo,0,80,Riesgo,S,17,acebeda-la-id28001
4,2024-08-29 16:00:00,Nubes altas,21,21,34.0,0.0,0,65,Riesgo,0,80,Riesgo,S,20,acebeda-la-id28001


Cielo is similar to category in the foursquare dataset. It provides qualitative value, but it is not worth separating it in a different table until there are specific characteristics to each cielo category.

In [464]:
aemet_df["cielo"].unique()

array(['Nubes altas', 'Muy nuboso',
       'Cubierto con tormenta y lluvia escasa', 'Nuboso',
       'Cielo despejado', 'Poco nuboso',
       'Intervalos nubosos con lluvia escasa', 'Cubierto',
       'Cubierto con lluvia escasa', 'Cubierto con lluvia',
       'Muy nuboso con tormenta y lluvia escasa', 'Muy nuboso con lluvia',
       'Intervalos nubosos', 'Muy nuboso con lluvia escasa',
       'Nuboso con tormenta y lluvia escasa', 'Cubierto con tormenta',
       'Nuboso con lluvia escasa', 'Intervalos nubosos con lluvia',
       'Bruma', 'Niebla', 'Nuboso con lluvia', 'Muy nuboso con tormenta',
       'Nuboso con tormenta'], dtype=object)

prob._precip.__(%) is also inconsistent, with mixed data types. What's more, looking up the source at Aemet's website, it registers data for a range of hours rather than hourly.

Avisos and prob._precip.__(%) are to be dropped before uploading them to the database.

Finally, localizacion_id is a perfect column to create as a common id to join by the tables in the database.

### 2.3 Municipios

Very straightforward in this case. This dataset could have the localizacion_id attached, to serve as the center of the database with foursquare and aemet at both sides.

In [465]:
municipios_df = pd.read_csv("../data/municipios.csv",index_col=0).T.reset_index()
municipios_df.columns = ["municipio_normalizado","latitude", "longitude","municipio"]
municipios_df

,municipio_normalizado,latitude,longitude,municipio
0,acebeda-la,41.0869583,-3.624399,La Acebeda
1,ajalvir,40.5342302,-3.4807818,Ajalvir
2,alameda-del-valle,40.9177178,-3.8438216,Alameda del Valle
3,alamo-el,40.2307296,-3.9905893,El Álamo
4,alcala-de-henares,40.4818396,-3.3644973,Alcalá de Henares
...,...,...,...,...
171,villar-del-olmo,40.3364465,-3.2355624,Villar del Olmo
172,villarejo-de-salvanes,40.1683582,-3.2738764,Villarejo de Salvanés
173,villaviciosa-de-odon,40.3573787,-3.9002334,Villaviciosa de Odón
174,villavieja-del-lozoya,41.0063622,-3.6716291,Villavieja del Lozoya


## 3. Transformation and upload

### 3.1 Transformation of tables

Let's start by the Aemet dataset, has it has the id that will be used to join all tables.

In [466]:
aemet_df[["municipio_normalizado","id_municipio"]] = aemet_df["localizacion_id"].str.extract(r'(.*)-(id\d{5})',expand=True)
aemet_df.drop(columns=["avisos","prob._precip.__(%)","localizacion_id"], inplace=True)
aemet_df.columns = ["fecha",  "cielo", "temp_celsius", "sen_termica_celsius", "racha_max_km_h", "precipitacion_mm", "nieve_mm", "humedad_relativa_pct", "prob_de_nieve_pct", "prob_de_tormenta_pct", "direccion_viento", "velocidad_viento","municipio_normalizado","id_municipio"]
aemet_df.head()


,fecha,cielo,temp_celsius,sen_termica_celsius,racha_max_km_h,precipitacion_mm,nieve_mm,humedad_relativa_pct,prob_de_nieve_pct,prob_de_tormenta_pct,direccion_viento,velocidad_viento,municipio_normalizado,id_municipio
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,0,75,S,7,acebeda-la,id28001
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,0,75,SE,12,acebeda-la,id28001
2,2024-08-29 14:00:00,Muy nuboso,23,23,33.0,0.0,0,59,0,80,S,14,acebeda-la,id28001
3,2024-08-29 15:00:00,Nubes altas,23,23,35.0,0.0,0,55,0,80,S,17,acebeda-la,id28001
4,2024-08-29 16:00:00,Nubes altas,21,21,34.0,0.0,0,65,0,80,S,20,acebeda-la,id28001


Now, the foursquare dataset can have:
- The id_municipio attached.
- The municipio variable dropped.
- The distance variable dropped.

In [467]:
municipio_id = aemet_df[["id_municipio","municipio_normalizado"]].value_counts().reset_index()

In [468]:
foursquare_df_id = foursquare_df.merge(municipio_id[["id_municipio","municipio_normalizado"]], left_on="municipio", right_on="municipio_normalizado", how="inner")
foursquare_df_id.drop(columns=["municipio_normalizado","municipio","distance"], inplace=True)
foursquare_df_id

,category,fsq_id,closed_bucket,link,name,address,latitude,longitude,id_municipio
0,Monument,03f82c6f2c134099b43faadc,Unsure,/v3/places/03f82c6f2c134099b43faadc,Sociedad Cooperativa Limitada el Encinar de Mi...,"Pza. de España, 1, Miraflores de la Sierra Madrid",40.813749,-3.766791,id28085
1,Monument,d52b434e8a2a45967eb0fcab,LikelyOpen,/v3/places/d52b434e8a2a45967eb0fcab,Casa Consistorial de Alcalá de Henares,"Plaza de Cervantes, 12, 28801 Alcalá de Henare...",40.481808,-3.364352,id28005
2,Park,4e481f77d4c065cb6bb5ecae,LikelyOpen,/v3/places/4e481f77d4c065cb6bb5ecae,El Parque,Comunidad de Madrid,40.673340,-4.089480,id28068
3,Monument,393a3d5770fc487b50c1ed69,LikelyOpen,/v3/places/393a3d5770fc487b50c1ed69,Semana Santa,28013 Madrid,40.500878,-4.238318,id28125
4,Park,fdbc8d360b264c50faedf5a8,Unsure,/v3/places/fdbc8d360b264c50faedf5a8,Diverarte,28750 San Agustín del Guadalix Comunidad de Ma...,40.679033,-3.616320,id28129
...,...,...,...,...,...,...,...,...,...
817,Park,4f534690e4b028a9204243d1,LikelyOpen,/v3/places/4f534690e4b028a9204243d1,Parque Canino Parque Asturias,Principado de Asturias,40.361813,-3.551802,id28123
818,Park,4dd6abc6d22d38ef42d5584a,VeryLikelyOpen,/v3/places/4dd6abc6d22d38ef42d5584a,Prado Grande,Prado Madrid,40.578151,-3.951944,id28152
819,Park,4e9eab5d6c25364827a58c88,VeryLikelyOpen,/v3/places/4e9eab5d6c25364827a58c88,Pinar de las Rozas,Pinar Comunidad de Madrid,40.507475,-3.890129,id28127
820,Park,4ecfa77ecc219860f67f592c,LikelyOpen,/v3/places/4ecfa77ecc219860f67f592c,Avd. del Mar Adriatico,NaN,40.175938,-3.687765,id28161


Finally, to add the id_municipio to the municipios table.

In [469]:
municipios_df = municipios_df.merge(municipio_id[["id_municipio","municipio_normalizado"]], on="municipio_normalizado", how="inner")
municipios_df


,municipio_normalizado,latitude,longitude,municipio,id_municipio
0,acebeda-la,41.0869583,-3.624399,La Acebeda,id28001
1,ajalvir,40.5342302,-3.4807818,Ajalvir,id28002
2,alameda-del-valle,40.9177178,-3.8438216,Alameda del Valle,id28003
3,alamo-el,40.2307296,-3.9905893,El Álamo,id28004
4,alcala-de-henares,40.4818396,-3.3644973,Alcalá de Henares,id28005
...,...,...,...,...,...
171,villar-del-olmo,40.3364465,-3.2355624,Villar del Olmo,id28179
172,villarejo-de-salvanes,40.1683582,-3.2738764,Villarejo de Salvanés,id28180
173,villaviciosa-de-odon,40.3573787,-3.9002334,Villaviciosa de Odón,id28181
174,villavieja-del-lozoya,41.0063622,-3.6716291,Villavieja del Lozoya,id28182


### 3.2 Upload

### 3.2.1 Connection to the database

In [470]:
connection = connect_to_database("modulo4_lab3",credentials_dict=database_credentials)
cursor = connection.cursor()

### 3.2.2 Creation of structure and tables

In [471]:

query_creation = """
CREATE TABLE IF NOT EXISTS municipios(
id_municipio VARCHAR(7) PRIMARY KEY,
municipio_normalizado VARCHAR(100) NOT NULL UNIQUE,
municipio VARCHAR(100) NOT NULL UNIQUE,
latitude DECIMAL,
longitude DECIMAL
);


CREATE TABLE IF NOT EXISTS foursquare_location(
fsq_id VARCHAR(200) PRIMARY KEY,
location_name VARCHAR (200) NOT NULL,
id_municipio VARCHAR(7),
closed_bucket VARCHAR(30),
category VARCHAR(100),
address VARCHAR(200),
latitude DECIMAL,
longitude DECIMAL,
link VARCHAR(200),
FOREIGN KEY (id_municipio) 
	REFERENCES municipios(id_municipio)
    ON DELETE RESTRICT
    ON UPDATE CASCADE
);

					
CREATE TABLE IF NOT EXISTS weather(
id_weather SERIAL PRIMARY KEY,
fecha TIMESTAMP NOT NULL,
id_municipio VARCHAR(7),
cielo VARCHAR(40), 
temp_celsius INT,
sen_termica_celsius INT,
racha_max_km_h DECIMAL,
precipitacion_mm DECIMAL,
nieve_mm DECIMAL (5,2),
humedad_relativa_pct DECIMAL,
prob_de_nieve_pct DECIMAL,
prob_de_tormenta_pct DECIMAL,
direccion_viento VARCHAR(2),
velocidad_viento INT,
FOREIGN KEY (id_municipio) 
	REFERENCES municipios(id_municipio)
    ON DELETE RESTRICT
    ON UPDATE CASCADE
);"""

cursor.execute(query_creation)
connection.commit()

### 3.2.3 Data load

##### 3.2.3.1 Municipio

In [472]:
aemet_df.head()

,fecha,cielo,temp_celsius,sen_termica_celsius,racha_max_km_h,precipitacion_mm,nieve_mm,humedad_relativa_pct,prob_de_nieve_pct,prob_de_tormenta_pct,direccion_viento,velocidad_viento,municipio_normalizado,id_municipio
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,0,75,S,7,acebeda-la,id28001
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,0,75,SE,12,acebeda-la,id28001
2,2024-08-29 14:00:00,Muy nuboso,23,23,33.0,0.0,0,59,0,80,S,14,acebeda-la,id28001
3,2024-08-29 15:00:00,Nubes altas,23,23,35.0,0.0,0,55,0,80,S,17,acebeda-la,id28001
4,2024-08-29 16:00:00,Nubes altas,21,21,34.0,0.0,0,65,0,80,S,20,acebeda-la,id28001


In [473]:
aemet_df[aemet_df["id_municipio"]=="id28030"].head(2)

,fecha,cielo,temp_celsius,sen_termica_celsius,racha_max_km_h,precipitacion_mm,nieve_mm,humedad_relativa_pct,prob_de_nieve_pct,prob_de_tormenta_pct,direccion_viento,velocidad_viento,municipio_normalizado,id_municipio
1278,2024-08-29 12:00:00,Muy nuboso,22,22,20.0,0.0,0,74,0,75,SE,6,cabrera-la,id28030
1279,2024-08-29 13:00:00,Nubes altas,23,23,25.0,0.0,0,67,0,75,SE,8,cabrera-la,id28030


In [474]:
municipios_order = ["id_municipio","municipio_normalizado","municipio","latitude","longitude"]

municipio_values = [tuple(value_row) for value_row in municipios_df[municipios_order].values]
municipio_values[:2]

[('id28001', 'acebeda-la', 'La Acebeda', '41.0869583', '-3.624399'),
 ('id28002', 'ajalvir', 'Ajalvir', '40.5342302', '-3.4807818')]

In [475]:
query_insercion = """INSERT INTO municipios (id_municipio,municipio_normalizado,municipio,latitude,longitude) VALUES (%s,%s,%s,%s,%s);"""
cursor.executemany(query_insercion, municipio_values) # cuando se quieren insertar varios
connection.commit()

##### 3.2.3.2 Foursquare_location

In [476]:
foursquare_order = ["fsq_id", "name", "id_municipio", "closed_bucket", "category", "address", "latitude", "longitude", "link"]

foursquare_values = [tuple(value_row) for value_row in foursquare_df_id[foursquare_order].values]
foursquare_values[0:2]

[('03f82c6f2c134099b43faadc',
  'Sociedad Cooperativa Limitada el Encinar de Miraflores',
  'id28085',
  'Unsure',
  'Monument',
  'Pza. de España, 1, Miraflores de la Sierra Madrid',
  40.813749,
  -3.766791,
  '/v3/places/03f82c6f2c134099b43faadc'),
 ('d52b434e8a2a45967eb0fcab',
  'Casa Consistorial de Alcalá de Henares',
  'id28005',
  'LikelyOpen',
  'Monument',
  'Plaza de Cervantes, 12, 28801 Alcalá de Henares Madrid',
  40.481808,
  -3.364352,
  '/v3/places/d52b434e8a2a45967eb0fcab')]

In [477]:
query_insercion = """INSERT INTO foursquare_location (fsq_id, location_name, id_municipio, closed_bucket, category, address, latitude, longitude, link
) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s);"""
cursor.executemany(query_insercion, foursquare_values) # cuando se quieren insertar varios
connection.commit()

##### 3.2.3.3 Weather

In [478]:
aemet_df = aemet_df[["fecha", "id_municipio", "cielo", "temp_celsius", "sen_termica_celsius", "racha_max_km_h", "precipitacion_mm", "nieve_mm", "humedad_relativa_pct", "prob_de_nieve_pct", "prob_de_tormenta_pct", "direccion_viento", "velocidad_viento"]]
aemet_df.head(2)

,fecha,id_municipio,cielo,temp_celsius,sen_termica_celsius,racha_max_km_h,precipitacion_mm,nieve_mm,humedad_relativa_pct,prob_de_nieve_pct,prob_de_tormenta_pct,direccion_viento,velocidad_viento
0,2024-08-29 12:00:00,id28001,Nubes altas,21,21,25.0,0.0,0,69,0,75,S,7
1,2024-08-29 13:00:00,id28001,Nubes altas,24,24,29.0,0.0,0,57,0,75,SE,12


In [479]:
aemet_values = [tuple(value_row) for value_row in aemet_df.values]
aemet_values

[(Timestamp('2024-08-29 12:00:00'),
  'id28001',
  'Nubes altas',
  21,
  21,
  25.0,
  0.0,
  0,
  69,
  0,
  75,
  'S',
  7),
 (Timestamp('2024-08-29 13:00:00'),
  'id28001',
  'Nubes altas',
  24,
  24,
  29.0,
  0.0,
  0,
  57,
  0,
  75,
  'SE',
  12),
 (Timestamp('2024-08-29 14:00:00'),
  'id28001',
  'Muy nuboso',
  23,
  23,
  33.0,
  0.0,
  0,
  59,
  0,
  80,
  'S',
  14),
 (Timestamp('2024-08-29 15:00:00'),
  'id28001',
  'Nubes altas',
  23,
  23,
  35.0,
  0.0,
  0,
  55,
  0,
  80,
  'S',
  17),
 (Timestamp('2024-08-29 16:00:00'),
  'id28001',
  'Nubes altas',
  21,
  21,
  34.0,
  0.0,
  0,
  65,
  0,
  80,
  'S',
  20),
 (Timestamp('2024-08-29 17:00:00'),
  'id28001',
  'Cubierto con tormenta y lluvia escasa',
  21,
  21,
  35.0,
  0.1,
  0,
  59,
  0,
  80,
  'S',
  17),
 (Timestamp('2024-08-29 18:00:00'),
  'id28001',
  'Nuboso',
  20,
  20,
  19.0,
  0.0,
  0,
  65,
  0,
  80,
  'S',
  20),
 (Timestamp('2024-08-29 19:00:00'),
  'id28001',
  'Nuboso',
  19,
  19,
  27

In [480]:
query_insercion = """INSERT INTO weather (fecha, id_municipio, cielo, temp_celsius, sen_termica_celsius, racha_max_km_h, precipitacion_mm, nieve_mm, humedad_relativa_pct, prob_de_nieve_pct, prob_de_tormenta_pct, direccion_viento, velocidad_viento
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
cursor.executemany(query_insercion, aemet_values) # cuando se quieren insertar varios
connection.commit()

ForeignKeyViolation: inserción o actualización en la tabla «weather» viola la llave foránea «weather_id_municipio_fkey»
DETAIL:  La llave (id_municipio)=(id28030) no está presente en la tabla «municipios».


It seems that there's an id that is not present in the municipios dataset. So let's check if the value is really there or not, if it is not, it should be dropped.

In [481]:
municipios_df[municipios_df["id_municipio"]=="id28030"]

,municipio_normalizado,latitude,longitude,municipio,id_municipio


It is indeed not there, so it should be dropped directly, as the municipio is not being monitored in the database.

In [482]:
lista_id_municipio = list(municipios_df["id_municipio"].unique())

aemet_df = aemet_df[aemet_df["id_municipio"].isin(lista_id_municipio)]

aemet_values = [tuple(value_row) for value_row in aemet_df.values]
aemet_values[:2]

[(Timestamp('2024-08-29 12:00:00'),
  'id28001',
  'Nubes altas',
  21,
  21,
  25.0,
  0.0,
  0,
  69,
  0,
  75,
  'S',
  7),
 (Timestamp('2024-08-29 13:00:00'),
  'id28001',
  'Nubes altas',
  24,
  24,
  29.0,
  0.0,
  0,
  57,
  0,
  75,
  'SE',
  12)]

In [483]:
connection = connect_to_database("modulo4_lab3",credentials_dict=database_credentials)
cursor = connection.cursor()

query_insercion = """INSERT INTO weather (fecha, id_municipio, cielo, temp_celsius, sen_termica_celsius, racha_max_km_h, precipitacion_mm, nieve_mm, humedad_relativa_pct, prob_de_nieve_pct, prob_de_tormenta_pct, direccion_viento, velocidad_viento)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
cursor.executemany(query_insercion, aemet_values) # cuando se quieren insertar varios
connection.commit()